In [ ]:
import json

# Open the JSON file
with open('./../Poems_And_Outputs/all_songs_trankit_output.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

datasongs = []
datalabels = []

for song in data:
  if not song['year'] == "":
    datasongs.append(song['content'])
    if int(song['year']) < 1900:
      datalabels.append(0)
    elif int(song['year']) < 1910:
      datalabels.append(1)
    else:
      datalabels.append(2)
print(len(datalabels))


106


In [ ]:
def word_to_list_of_part_of_speech(word):
  ans = []
  if word.get('upos') is not None:
    ans.append(word['upos'])
    return ans
  subwords = word['expanded']
  for subword in subwords:
    ans.extend(word_to_list_of_part_of_speech(subword))
  return ans



def song_to_list_of_part_of_speech(sentences):
  ans = []
  for part in sentences:
    tokens = part['tokens']
    for word in tokens:
      ans.extend(word_to_list_of_part_of_speech(word))
  return ans


print(song_to_list_of_part_of_speech(data[0]['content']['sentences']))

['PROPN', 'ADJ', 'NOUN', 'PRON', 'PUNCT', 'NOUN', 'ADJ', 'PUNCT', 'ADP', 'NOUN', 'DET', 'NOUN', 'ADP', 'ADP', 'NOUN', 'PUNCT', 'ADP', 'NOUN', 'ADP', 'PRON', 'SCONJ', 'NOUN', 'ADV', 'VERB', 'NOUN', 'NOUN', 'ADP', 'PRON', 'ADP', 'NOUN', 'X', 'VERB', 'ADP', 'NOUN', 'PUNCT', 'PROPN', 'PUNCT', 'NOUN', 'PUNCT', 'ADJ', 'DET', 'VERB', 'PUNCT', 'NOUN', 'NOUN', 'ADJ', 'PUNCT', 'ADV', 'ADV', 'ADP', 'DET', 'NOUN', 'DET', 'ADJ', 'PUNCT', 'DET', 'ADJ', 'PUNCT', 'VERB', 'DET', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'PRON', 'PROPN', 'ADP', 'PROPN', 'ADP', 'PROPN', 'PUNCT', 'ADV', 'DET', 'NOUN', 'DET', 'ADJ', 'PUNCT', 'PROPN', 'VERB', 'PUNCT', 'VERB', 'NOUN', 'SCONJ', 'VERB', 'PUNCT', 'CCONJ', 'VERB', 'ADP', 'VERB', 'PUNCT', 'VERB', 'VERB', 'ADV', 'VERB', 'ADV', 'NOUN', 'PUNCT', 'ADV', 'VERB', 'PUNCT', 'DET', 'VERB', 'ADP', 'PRON', 'VERB', 'PUNCT', 'ADJ', 'PUNCT', 'ADJ', 'PUNCT', 'NOUN', 'ADP', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'NOUN', 'PUNCT', 'DET', 'NOUN',

In [ ]:
allSongsByPartOfSpeech = []

for song in datasongs:
  allSongsByPartOfSpeech.append(song_to_list_of_part_of_speech(song['sentences']))

print(len(allSongsByPartOfSpeech))

106


In [ ]:
typesPartOfSpeech = []

for song in allSongsByPartOfSpeech:
  for part in song:
    if part not in typesPartOfSpeech:
      typesPartOfSpeech.append(part)

print(typesPartOfSpeech)

['PROPN', 'ADJ', 'NOUN', 'PRON', 'PUNCT', 'ADP', 'DET', 'SCONJ', 'ADV', 'VERB', 'X', 'CCONJ', 'NUM', 'AUX']


In [ ]:
allSongsByPartOfSpeechIndex = []
for song in allSongsByPartOfSpeech:
  songByIndex = []
  for part in song:
    songByIndex.append(typesPartOfSpeech.index(part))
  allSongsByPartOfSpeechIndex.append(songByIndex)

print(len(allSongsByPartOfSpeechIndex))

106


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Hyperparameters
padding_value = 16
test_size = 0.2
random_state = 42

# Sample data: list of lists of integers
data = allSongsByPartOfSpeechIndex
labels = datalabels

# Find the maximum length of the lists
max_length = max(len(x) for x in data)

# Pad the lists with 16s to make them of equal length
padded_data = [x + [padding_value] * (max_length - len(x)) for x in data]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(padded_data, labels, test_size=test_size, random_state=random_state)

# Convert lists to numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Initialize Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)



Accuracy: 0.5909090909090909


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Hyperparameters
padding_value = 16
test_size = 0.2
random_state = 42
epochs = 10
batch_size = 32

# Sample data: list of lists of integers
data = allSongsByPartOfSpeechIndex
labels = datalabels

# Find the maximum length of the lists
max_length = max(len(x) for x in data)

# Pad the lists with 16s to make them of equal length
padded_data = [x + [padding_value] * (max_length - len(x)) for x in data]

# Convert lists to numpy arrays
X = np.array(padded_data)
y = np.array(labels)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

# Build the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=max_length+1, output_dim=16, input_length=max_length),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

# Predict on the test set
y_pred = np.argmax(model.predict(X_test), axis=-1)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Epoch 1/10
3/3 [==============================] - 6s 2s/step - loss: 1.0987 - accuracy: 0.4133 - val_loss: 1.0988 - val_accuracy: 0.3333
Epoch 2/10
3/3 [==============================] - 3s 870ms/step - loss: 1.0904 - accuracy: 0.3867 - val_loss: 1.0993 - val_accuracy: 0.3333
Epoch 3/10
3/3 [==============================] - 3s 878ms/step - loss: 1.0832 - accuracy: 0.4267 - val_loss: 1.1005 - val_accuracy: 0.3333
Epoch 4/10
3/3 [==============================] - 3s 894ms/step - loss: 1.0753 - accuracy: 0.4267 - val_loss: 1.1025 - val_accuracy: 0.3333
Epoch 5/10
3/3 [==============================] - 3s 895ms/step - loss: 1.0683 - accuracy: 0.4267 - val_loss: 1.1057 - val_accuracy: 0.3333
Epoch 6/10
3/3 [==============================] - 4s 1s/step - loss: 1.0603 - accuracy: 0.4267 - val_loss: 1.1111 - val_accuracy: 0.3333
Epoch 7/10
3/3 [==============================] - 3s 890ms/step - loss: 1.0509 - accuracy: 0.4267 - val_loss: 1.1207 - val_accuracy: 0.3333
Epoch 8/10
3/3 [==========

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Hyperparameters
padding_value = 16
test_size = 0.2
random_state = 42
epochs = 10
batch_size = 32
num_layers = 50

# Sample data: list of lists of integers
data = allSongsByPartOfSpeechIndex
labels = datalabels

# Find the maximum length of the lists
max_length = max(len(x) for x in data)

# Pad the lists with 16s to make them of equal length
padded_data = [x + [padding_value] * (max_length - len(x)) for x in data]

# Convert lists to numpy arrays
X = np.array(padded_data)
y = np.array(labels)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

# Build the deep neural network model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(max_length,)))
for _ in range(num_layers):
    model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

# Predict on the test set
y_pred = np.argmax(model.predict(X_test), axis=-1)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)




Epoch 1/10
3/3 [==============================] - 10s 280ms/step - loss: 1.0978 - accuracy: 0.3467 - val_loss: 1.0987 - val_accuracy: 0.3333
Epoch 2/10
3/3 [==============================] - 0s 27ms/step - loss: 1.0913 - accuracy: 0.4267 - val_loss: 1.0994 - val_accuracy: 0.3333
Epoch 3/10
3/3 [==============================] - 0s 38ms/step - loss: 1.0824 - accuracy: 0.4133 - val_loss: 1.1012 - val_accuracy: 0.3333
Epoch 4/10
3/3 [==============================] - 0s 26ms/step - loss: 1.0715 - accuracy: 0.4133 - val_loss: 1.1080 - val_accuracy: 0.3333
Epoch 5/10
3/3 [==============================] - 0s 28ms/step - loss: 1.0525 - accuracy: 0.4133 - val_loss: 1.1388 - val_accuracy: 0.3333
Epoch 6/10
3/3 [==============================] - 0s 34ms/step - loss: 1.0287 - accuracy: 0.4267 - val_loss: 1.2774 - val_accuracy: 0.3333
Epoch 7/10
3/3 [==============================] - 0s 25ms/step - loss: 1.0406 - accuracy: 0.4267 - val_loss: 1.2180 - val_accuracy: 0.3333
Epoch 8/10
3/3 [=========